# Classic: Penny-Pinching

The full [riddle](https://fivethirtyeight.com/features/how-many-pennies-should-you-pinch/).

> The game starts with somewhere between 20 and 30 pennies, which I then divide into two piles any way I like. Then we alternate taking turns, with you first, until someone wins the game. For each turn, a player may take any number of pennies he or she likes from either pile, or instead take the same number of pennies from both piles. Each player must also take at least one penny every turn. The winner of the game is the one who takes the last penny.

> If we both play optimally, what starting numbers of pennies (again, between 20 and 30) guarantee that you can win the game?

You'd think the "dealer" has the advantage by controlling the number of pennies in each pile. Also, the phrase "somewhere between" might be a give-away in that it really doesn't matter how many pennies are in the game. Let's play along.

Take a simple case of 4 pennies, pile A with 3 and pile B with 1. I would take one from A and win the game because:
* If you take one from A, I take the last one from each and end the game
* If you take one from B, I take both from A and end the game
* If you take one each from A and B, I take the last one from A and end the game
* If you take two from A, I take the last one from B and end the game

Be it resolved: if I can get the piles to 1 and 2, I win. This means you want to avoid a pile of 1 at all costs, because my next move will be to take all but two from the other pile.

Somethings so obvious that I almost didn't mention them:
* Equal piles are an instant win for the next player too. They will take 'em all.
* All coins in one pile is an instant win for me.

Now try 5 pennies. 4 in A and 1 in B is an instant win: I will take 2 from A. Now consider 3 in A and 2 in B. I will take one from each and win.

I'm starting to get the feeling that I might always win. Let's just throw some random boards out there and see.

* 0000000
* 000

My turn

* 00000
* 000

Your turn
* 0000
* 000

That was over fast. Why? Can't tell yet, let's do another (we'll use numbers this time).

* 10
* 4

My turn

* 8
* 4

Your turn

* 7
* 4

My turn

* 6
* 4

Your turn

* 5
* 3

Over again I think, but you won, again at 5 in A and 3 in B.

I think I got it but can't prove it yet. If we call the size of the piles $A$ and $B$ where $A > B$, then my job is to keep $A = 2B-1$ when $B \geq 3$. When $A-B=1$, reduce until $A=2$ and $B=1$. If either pile has two or one, make the other pile have one or two, respectively. Then I don't think it matters what you do. Let me see with something that falls in range of the problem.

* 15
* 11

My turn

* 6
* 11

Your turn

* 6
* 9

My turn

* 4
* 7

Your turn

* 4
* 6

My turn

* 5
* 3

But of course! If you are in charge of splitting the pile and can control the starting number of coins, then you would select a starting number of coins that can initially be split into the $A=2B-1$ scheme right away. Then, since the rules force me to take at least one coin per turn, I'll disrupt this pattern, but you only need to keep it in place for the rest of the game to win.

Which starting values between 20 and 30 fit this pattern? This is a system of equations. Let $T$ be the total.
$$A = 2B-1$$
$$A+B=T$$

In [1]:
import numpy as np
for i in range(20, 31):
    e1 = np.array([[1, -2], [1, 1]])
    e2 = np.array([-1, i])
    s = np.linalg.solve(e1, e2)
    print(s)

[13.  7.]
[13.66666667  7.33333333]
[14.33333333  7.66666667]
[15.  8.]
[15.66666667  8.33333333]
[16.33333333  8.66666667]
[17.  9.]
[17.66666667  9.33333333]
[18.33333333  9.66666667]
[19. 10.]
[19.66666667 10.33333333]


I think I'm missing something important, because it appears to work for every third value, or all $3N-1$ for $N\geq2$. Regardless, since totals of 20, 23, 26, and 29 solve to whole values, any one of these numbers will ensure that you always win, provided you split them as described by the equations. Conversely, if you select any other starting total, I will win. What a neat puzzle!

This just feels correct to me. I'll leave it to someone else to mathematically prove it. I could build an optimal simulator but I don't know how to build a non-optimal player. Random? Me? This would be a start.

## Addendum

Dead wrong. What's so aggravating is that a counterexample to my solution was right in front of me: piles (5, 9). This fits my pattern, but of course if you split this way, I will make the piles (5, 3) and win.

The pattern is more beautiful than that, and it does hold for the first few arrangements that my solution also predicted: (1,2), (3,5), (4,7). Then it breaks: (6, 10), (8, 13), (9, 15). How would you arrive at this, without knowing the pattern in advance?

First, identify the base case. I got that far. Then I should have done more simulations, programmatically or otherwise. I think if I had just identified the (6, 10) case, I would have seen the pattern. And instead of building a simulator with players, a state machine would have been more appropriate.



But without computers, I think one can still arrive at a solution. After you get the base cases, it becomes apparent that players need to avoid a move that makes the difference between two piles the same as any lower winning configuration. E.g. if I move to make the piles (15, 18), you can move to (3, 5) and win. Then there's another, more subtle winning rule: you can't move to a state where only one pile shares a value with a winning combo.

It's knowing these rules of thumb that makes it possible to start generating the winning piles (from the dealer's perspective). We start with (1,2). Now we know there can't be another pile with a difference of 1, otherwise I could reduce them to (1,2), so the next winning pile should be separated by 2. How about (2, 4)? No good, because it contains a pile of 2, which is already part of a winning pile. Thus we have (3, 5). Difference of 3, (4,7), difference of 4, we skip (5,9) since 5 appeared earlier and move to (6, 10).

For fun, let's look at the ratio of the piles. If you're like me your gut is telling you it should be converging to something.

In [2]:
number_set = set([1, 2])
a, b = (1,2)
for i in range (1000000):
    diff = b - a + 1
    a += 1
    while a in number_set:
        a += 1
    b = a + diff
    if i % 100000 == 0:
        print(b / a)
    number_set.add(a)
    number_set.add(b)

1.6666666666666667
1.618036413977232
1.6180340533358055
1.618034539660042
1.6180347828236632
1.6180341647919705
1.6180343893789084
1.6180340041337529
1.6180341926562272
1.6180343392849907


The golden ratio. Notice though, it's taking a helluva long time to converge.